# 📦 Step 1: Install Required Packages

Before you start using this notebook, make sure all the necessary Python packages are installed. These packages are listed in the `requirements.txt` file.

## 💻 How to Install

Open your terminal or command prompt, navigate to the project directory (where `requirements.txt` is located), and run:

```bash
pip install -r requirements.txt
```

This command will automatically install all dependencies needed for the notebook and project to run smoothly.

> ✅ **Tip:** If you're using a virtual environment (recommended), make sure it's activated before running the command.

## 🛠️ Troubleshooting

* If you get a "permission denied" error, try adding `--user`:

  ```bash
  pip install --user -r requirements.txt
  ```

* If you're working in Jupyter and want to install directly from a notebook cell, you can run:

  ```python
  !pip install -r requirements.txt
  ```

# 🔧 Second Step: Create and Register Your Project Folder

In this step, you'll register the **main project folder**, so the system knows where to store and retrieve all files related to your project.

This setup automatically creates a structured folder system under your specified `main_folder`, and stores the configuration in a central JSON file. This ensures your projects remain organized, especially when working with multiple studies or datasets.

---

## 🗂️ Project Folder Structure

On the first run, the following folder structure will be automatically created under your specified `main_folder`:

```
main_folder/
├── database/
│   └── scopus/
│   └── <project_name>_database.xlsx  ← auto-generated path (not file creation)
├── pdf/
└── xml/
```

---

## 💡 Example

Suppose your project name is `corona_discharge`, and you want to store all project files under:

```
G:\My Drive\research_related\ear_eog
```

You can register this setup by running:

```python
project_folder(
    project_review='corona_discharge',
    main_folder=r'G:\My Drive\research_related\ear_eog'
)
```

✅ This will:

* Save the project path in `setting/project_folders.json`
* Create the full folder structure: `database/scopus`, `pdf`, and `xml`

---

## 🔁 Loading the Project Later

Once registered, you can access the project folders in future sessions by providing just the `project_review` name:

```python
paths = project_folder(project_review='corona_discharge')

print(paths['main_folder'])  # Main project folder
print(paths['csv_path'])     # Path to the Excel database file
```

---

## ⚙️ What Happens in the Background?

* A file named `project_folders.json` is stored in the `setting/` directory within your project.
* It maps each `project_review` to its corresponding `main_folder`.
* Folder structure is created automatically on the first run.
* On subsequent runs, the system reads the JSON to locate your project — no need to re-enter paths.


In [6]:
from setting.project_path import project_folder
project_name='corona_discharge'
main_folder=r"D:\my_project"

project_folder(project_review=project_name, main_folder=main_folder)

{'main_folder': 'D:\\my_project',
 'csv_path': 'D:\\my_project\\database\\corona_discharge_database.xlsx'}

# 📥 Third Step: Download the Scopus BibTeX File

In this step, you'll use the **Scopus database** to find and download relevant papers for your project. Scopus is a comprehensive and widely-used repository of peer-reviewed academic literature.

---

## 🔍 Using Scopus Advanced Search

To retrieve high-quality and relevant papers, we recommend using **Scopus' Advanced Search** feature. This powerful tool lets you refine your search based on:

* Keywords
* Authors
* Publication dates
* Document types
* And more...

This ensures that your literature collection is both targeted and comprehensive.

---

## 💡 Get Keyword Ideas with a Prompt

To help you formulate effective search queries, you can use the following **prompt-based suggestion tool**:

👉 [Keyword Search Prompt](https://gist.github.com/balandongiv/886437963d38252e61634ddc00b9d983)

You may need to modify the prompt to better suit your research domain. Here are some example domains:

* `"corona discharge"`
* `"fatigue driving EEG"`
* `"wafer classification"`

Feel free to add, remove, or tweak keywords as needed to refine your search results.

---

## 💾 Save and Organize Your Results

Once you've finalized your search:

1. **Select all available attributes** when exporting results from Scopus.
2. # Access Scopus
![Scopus CSV Export](../image/scopus_csv_export.png)


2. Choose the **BibTeX** format when saving the export file.
3. Save the file inside the `database/scopus/` folder of your project.

The resulting folder structure might look like this:

```
main_folder/
├── database/
│   └── scopus/
│       ├── scopus(1).bib
│       ├── scopus(2).bib
│       ├── scopus(3).bib
```

Make sure the BibTeX files are correctly named and stored to ensure smooth integration in later steps.

# 📊 Fourth Step: Combine Scopus BibTeX Files into Excel

Once you've downloaded multiple `.bib` files from Scopus, the next step is to **combine and convert** them into a structured Excel file. This makes it easier to filter, sort, and review the metadata of all collected papers.

---

## 🧰 What This Step Does

* Loads all `.bib` files from your project's `database/scopus/` folder
* Parses the relevant metadata (e.g., title, authors, year, source, DOI)
* Combines the results into a single Excel spreadsheet
* Saves the spreadsheet in the `database/` folder as `combined_filtered.xlsx`



## 📁 Folder Structure Example

After running the script, your folder might look like this:

```
main_folder/
├── database/
│   ├── scopus/
│   │   ├── scopus(1).bib
│   │   ├── scopus(2).bib
│   │   ├── scopus(3).bib
│   └── combined_filtered.xlsx
```

This Excel file will serve as your primary reference for filtering papers before downloading PDFs.


In [1]:
import os
from download_pdf.database_preparation import combine_scopus_bib_to_excel
from setting.project_path import project_folder

project_review='corona_discharge'
path_dic=project_folder(project_review=project_review)
folder_path=path_dic['scopus_path']
output_excel =  path_dic['database_path']
combine_scopus_bib_to_excel(folder_path, output_excel)

Found 3 .bib files in D:\my_project\database\scopus
Initial number of rows: 7
Number of rows after duplicate removal: 5. Total duplicates removed: 2
These are the unique publisher_long values: ['Nature Research' 'Springer Science and Business Media B.V.'
 'BioMed Central Ltd']
['nature' 'springer' 'biomedcentral']
Combined file has been saved to: D:\my_project\database\corona_discharge_database.xlsx


# 🧾 **Sixth Step: Download PDFs**

This step automates downloading PDFs for filtered academic papers using Selenium. The script uses metadata from `combined_filtered.xlsx` and saves each PDF with a filename based on its **BibTeX citation key**, ensuring consistent and human-readable identification.

> 🛑 **Note:** This process launches a real browser window; some publishers block headless downloads.

> 📝 **Important:** Only **Sci-Hub** is enabled by default. To activate fallback downloads from specific sources (e.g., IEEE, MDPI, ScienceDirect), you must **manually uncomment the corresponding function calls** in the script. This gives you full control over which fallback methods are used.

---

## ✨ What This Step Does

* Loads filtered paper metadata from Excel (`combined_filtered.xlsx`).
* Attempts to download PDFs from **Sci-Hub**.
* Falls back to other publishers (IEEE, MDPI, ScienceDirect) **if you uncomment those lines**.
* **Saves each PDF as `bibtex_key.pdf`** for easy referencing.

---

## 🧰 Code Snippet

```python
from download_pdf.download_pdf import (
    run_pipeline,
    process_scihub_downloads,
    process_fallback_ieee,
    # process_fallback_ieee_search,
    # process_fallback_mdpi,
    # process_fallback_sciencedirect,
)
from setting.project_path import project_folder
import os

# Project setup
project_review = 'wafer_defect'
path_dic = project_folder(project_review=project_review)
main_folder = path_dic['main_folder']

# Load filtered Excel file
file_path = path_dic['csv_path']
output_folder = os.path.join(main_folder, 'pdf')

# Load data and categorize by source
categories, data_filtered = run_pipeline(file_path)

# Step 1: Always try downloading from Sci-Hub first
process_scihub_downloads(categories, output_folder, data_filtered)

# Optional fallbacks — uncomment to enable each one:
# Step 2: IEEE fallback
# process_fallback_ieee(categories, data_filtered, output_folder)

# Step 3: IEEE Search fallback
# process_fallback_ieee_search(categories, data_filtered, output_folder)

# Step 4: MDPI fallback
# process_fallback_mdpi(categories, data_filtered, output_folder)

# Step 5: ScienceDirect fallback (may not work due to security restrictions)
# process_fallback_sciencedirect(categories, data_filtered, output_folder)

# Optionally save the updated Excel
# save_data(data_filtered, file_path)
```

---

## 🗂️ Folder Structure Example

```
wafer_defect/
├── database/
│   └── scopus/
│       └── combined_filtered.xlsx     ← Filtered metadata with BibTeX keys
├── pdf/
│   ├── smith_2020.pdf                 ← Saved using bibtex_key
│   ├── kim_2019.pdf
│   └── ...
```

In [ ]:
import os

from download_pdf.download_pdf import run_pipeline, process_scihub_downloads, process_fallback_ieee
from setting.project_path import project_folder

# Define your project
project_review = 'corona_discharge'

# Load project paths
path_dic = project_folder(project_review=project_review)
main_folder = path_dic['main_folder']

file_path = path_dic['database_path']
output_folder = os.path.join(main_folder, 'pdf')

# Run the main pipeline to load and categorize the data
categories, data_filtered = run_pipeline(file_path)

# First step, we will always use Sci-Hub to attempt PDF downloads
process_scihub_downloads(categories, output_folder, data_filtered)

# Fallback options for entries not available via Sci-Hub:
# Uncomment the following lines one by one if you want to try downloading from specific sources

# Uncomment to attempt fallback download from IEEE
# process_fallback_ieee(categories, data_filtered, output_folder)

# Uncomment to attempt fallback download using IEEE Search
# process_fallback_ieee_search(categories, data_filtered, output_folder)

# Uncomment to attempt fallback download from MDPI
# process_fallback_mdpi(categories, data_filtered, output_folder)

# Uncomment to attempt fallback download from ScienceDirect
# Note: ScienceDirect URLs can be extracted but PDFs may not be downloadable due to security restrictions
# process_fallback_sciencedirect(categories, data_filtered, output_folder)

# Uncomment to save the updated data to Excel after processing
# save_data(data_filtered, file_path)

# 🧾 **Seventh Step: Convert PDFs to XML using GROBID (OPTIONAL)**

After downloading the PDFs, the next step is to convert them into structured **TEI XML** format using [**GROBID**](https://grobid.readthedocs.io). This step enables downstream tasks like metadata extraction, reference parsing, and full-text analysis.

---

## 🧰 What This Step Does

* Processes all PDF files in your `pdf/` directory.
* Uses GROBID's **batch processing API**.
* Saves the resulting XML files into the `xml/` folder (one `.xml` per `.pdf`).
* Leverages Docker for fast, isolated execution.

---

## ⚙️ Setup Requirements

> 🛠️ **GROBID requires WSL + Docker on Windows**

* You must have **WSL** installed (tested on **WSL2 with Ubuntu 22.04**).
* You must have **Docker** installed and **running** before launching GROBID.

---

## 🐳 How to Install & Run GROBID

1. **Pull the Docker image from Docker Hub**
   Check for the [latest version](https://hub.docker.com/r/grobid/grobid/tags), or use the stable one:

   ```bash
   docker pull grobid/grobid:0.8.1
   ```

2. **Start the GROBID container in Ubuntu (WSL)**

   Open your Ubuntu terminal and run:

   ```bash
   docker run --rm --gpus all --init --ulimit core=0 -p 8070:8070 grobid/grobid:0.8.1
   ```

   > ✅ This exposes GROBID's REST API on `http://localhost:8070/`

3. **Test it in your browser**

   Open a browser (e.g., Firefox or Chrome) and navigate to:

   ```
   http://localhost:8070/
   ```

   You should see the GROBID interface.

---

## 🚀 Batch Conversion Command

Once the GROBID service is running, you can convert all PDFs in your `pdf/` folder to XML using:

```bash
# From your project root (in WSL/Ubuntu)
cd path/to/your/project

# Create output folder if not exists
mkdir -p xml

# Run batch processing using curl
curl -v --form "input=@pdf/" localhost:8070/api/processFulltextDocument -o xml/
```

Or, use a Python wrapper or script to iterate over PDFs and call GROBID’s REST API for more control.

---

## 🗂️ Folder Structure After Conversion

```
wafer_defect/
├── pdf/
│   ├── smith_2020.pdf
│   └── kim_2019.pdf
├── xml/
│   ├── smith_2020.xml
│   └── kim_2019.xml
```


# 🧾 **Eighth Step (Optional): Convert XML to JSON**

This step converts GROBID-generated TEI XML files into structured JSON format. While optional, it can be helpful for reviewing document content, integrating into other tools, or preparing data to feed into an LLM.

> 📝 **Note:** This step is **optional** — the main pipeline (`run_llm`) reads directly from XML. Use this conversion if you want to inspect or process JSON files instead.

---

## ✨ What This Step Does

* Reads all `*.xml` files from the `xml/` directory.
* Converts each into a corresponding `*.json` file (preserving the **BibTeX key as filename** for consistency).
* Stores all JSON outputs in `xml/json/`.

In addition, it handles and organizes special cases:

* 📁 **`xml/json/no_intro_conclusion/`**: XML files where GROBID could not detect an *introduction* or *conclusion* section.
* 📁 **`xml/json/untitled_section/`**: XML files where GROBID could not detect any section titles at all — these require manual checking.
* 📄 Other successfully processed files are stored directly in `xml/json/`.

---

## ▶️ Example Code

```python
from setting.project_path import project_folder
from grobid_tei_xml.xml_json import run_pipeline

project_review = 'corona_discharge'

# Load project paths
path_dic = project_folder(project_review=project_review)

# Convert all XML files in the specified folder to JSON
run_pipeline(path_dic['xml_path'])
```

---

## 🗂️ Example Folder Structure After Conversion

```
corona_discharge/
├── pdf/
│   ├── smith_2020.pdf
│   └── kim_2019.pdf
├── xml/
│   ├── smith_2020.xml
│   ├── kim_2019.xml
│   └── json/
│       ├── smith_2020.json
│       ├── kim_2019.json
│       ├── no_intro_conclusion/
│       │   └── failed_paper1.json
│       └── untitled_section/
│           └── failed_paper2.json
```

In [ ]:
from setting.project_path import project_folder
from grobid_tei_xml.xml_json import run_pipeline

project_review = 'corona_discharge'

# Load project paths
path_dic = project_folder(project_review=project_review)
run_pipeline(path_dic['xml_path'])

## 🧾 **Tenth Step: Export Filtered Excel to BibTeX**

After reviewing and filtering your `combined_filtered.xlsx` file, you can convert the refined list of papers back into a **BibTeX file**. This can be helpful for citation management or integration with tools like LaTeX or reference managers.

---

## ✨ What This Step Does

* Loads the filtered Excel file containing your selected papers
* Converts the data back into BibTeX format
* Saves the result to a `.bib` file for easy reuse or citation


## 📁 File Structure Example

```
bib_example/
├── combined_filtered.xlsx      ← Filtered Excel file with selected papers
└── filtered_output.bib         ← Newly generated BibTeX file
```

---

### 🛠️ Notes

* Ensure the Excel file has standard bibliographic columns like: `title`, `author`, `year`, `journal`, `doi`, etc.
* The function `generate_bibtex()` maps these fields into valid BibTeX entries.
* You can open the `.bib` file in any text editor or reference manager to confirm the results.

In [11]:
import os
import pandas as pd
from post_code_saviour.excel_to_bib import generate_bibtex
from setting.project_path import project_folder

# Define your project
project_review = 'corona_discharge'

# Load project paths
path_dic = project_folder(project_review=project_review)
main_folder = path_dic['main_folder']

# Define input and output paths
input_excel = os.path.join(main_folder, 'database', 'combined_filtered.xlsx')
output_bib = os.path.join(main_folder, 'database', 'filtered_output.bib')

# Load the filtered Excel file
df = pd.read_excel(input_excel)

# Generate BibTeX file
generate_bibtex(df, output_file=output_bib)


BibTeX file generated: D:\my_project\database\filtered_output.bib
